# Pittsburgh's Next Top Neighborhood

## 🚗 Commutability

In [1]:
import pandas as pd

# read census data
data = pd.read_csv("data/SNAP/Neighborhoods_with_SNAP_data.csv")

# make easier column names
data['name'] = data['Neighborhood_2010_HOOD']             # neighborhood name
data['drive'] = data['SNAP_All_csv_Commute_to_Work__D']   # drive to work alone
data['carpool'] = data['SNAP_All_csv_Commute_to_Work__C'] # carpool/vanpool
data['public'] = data['SNAP_All_csv_Commute_to_Work__P']  # public transit
data['taxi'] = data['SNAP_All_csv_Commute_to_Work__T']    # taxi
data['motor'] = data['SNAP_All_csv_Commute_to_Work__M']   # motorcycle
data['bike'] = data['SNAP_All_csv_Commute_to_Work__B']    # bike
data['walk'] = data['SNAP_All_csv_Commute_to_Work__W']    # walk
data['other'] = data['SNAP_All_csv_Commute_to_Work__O']   # other

# clean data
data = data.fillna(0)

# constants to determine value of method
# best values are 1, okay are 0.5, bad or null are 0.25
good = 1   # bike, walk
okay = 0.5 # carpool, public transit
bad = 0.25 # drive, taxi, motorcycle, other

# convert percent value to float
def convert(value):
    value = str(value)
    value = value.replace('%','')
    return float(value)

# iterate through rows, calculate commutability score
for idx, row in data.iterrows():
    score = convert(data.loc[idx,'bike']) + convert(data.loc[idx, 'walk']) # good methods
    score += okay*(convert(data.loc[idx, 'carpool']) + convert(data.loc[idx, 'public'])) # okay methods
    score += bad*(convert(data.loc[idx, 'drive']) + convert(data.loc[idx, 'taxi']) + convert(data.loc[idx, 'motor']) + convert(data.loc[idx, 'other'])) # bad methods
    data.at[idx,'raw'] = score # set neighborhood's commutability score


# find score using Pawel's formula
data['score'] = data['raw']/data['raw'].max()

# view results
commute = data[['name','score']]
commute = commute.sort_values(by=['score'], ascending=False)
commute = commute.reset_index(drop = True)
commute['rank'] = commute.index
commute = commute.sort_values(by=['name'], ascending=True).set_index('name')
commute

,score,rank
name,,
Allegheny Center,0.836108,2
Allegheny West,0.698008,7
Allentown,0.480248,54
Arlington,0.405158,78
Arlington Heights,0.616716,12
...,...,...
Upper Lawrenceville,0.464577,60
West End,0.433562,71
West Oakland,0.801828,5


In [4]:
data['name'] = data['Neighborhood_2010_HOOD']    
data['score'] = (-1)* data['SNAP_All_csv_Part_1_Crime_per_1']/(data['SNAP_All_csv_Part_1_Crime_per_1'].max())

data['score'] = (((1)* data['SNAP_All_csv_Part_1_Crime_per_1']/(data['SNAP_All_csv_Part_1_Crime_per_1'].max())) + ((0.5)* data['SNAP_All_csv_Part_2_Crime_per_1']/(data['SNAP_All_csv_Part_2_Crime_per_1'].max()))) / data['Pop__2010'] * 10 
data['score'] = data['score']/data['score'].max() 
crime = data[['name','score']].sort_values(by ='score',ascending=True)

#sort alphabetically 
#crime = crime.sort_values(by=['score']) # sort and set name index
#crime = crime.reset_index(drop = True)

#crime

# view results
crime = data[['name','score']]
crime = crime.sort_values(by=['score'], ascending=False)
crime = crime.reset_index(drop = True)
crime['rank'] = crime.index
crime = crime.sort_values(by=['name'], ascending=True).set_index('name')
crime






,score,rank
name,,
Allegheny Center,0.000679,11
Allegheny West,0.001283,8
Allentown,0.000197,29
Arlington,0.000113,45
Arlington Heights,0.002646,6
...,...,...
Upper Lawrenceville,0.000107,47
West End,0.003189,3
West Oakland,0.000077,56


In [7]:
final = data[['name']].copy()
final = final.sort_values(by='name').set_index('name')
final['commute_rank'] = commute['rank']# commute ranking 
final['crime_rank'] = crime['rank']# crime ranking
#add here for your datasets
final




,commute_rank,crime_rank
name,,
Allegheny Center,2,11
Allegheny West,7,8
Allentown,54,29
Arlington,78,45
Arlington Heights,12,6
...,...,...
Upper Lawrenceville,60,47
West End,71,3
West Oakland,5,56
